# Data

Currently the model's accuracy is hovering between 50-60%, it's crucial to note that it successfully validates the underlying concept. This initial success provides a solid base for subsequent development and fine-tuning.

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

df= pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')
df.sample(5)


,type,artist,song_title,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence
544,hiphop,Bliss n Eso,Tear The Roof Off (feat. Watsky),59,0.1610,0.736,206244,0.8900,0.000000,0.1660,-5.070,0,0.2490,142.101,0.679
404,hiphop,Young Scooter,No Features (feat. Kodak Black),59,0.0435,0.826,208287,0.7580,0.000000,0.1680,-5.616,1,0.0743,119.998,0.410
545,hiphop,Lil Tracy,This Year,57,0.0718,0.723,120000,0.5300,0.000000,0.1260,-9.754,0,0.2920,160.008,0.237
854,indie,Kina Grannis,Can’t Help Falling in Love,77,0.9050,0.266,201933,0.0596,0.000071,0.1320,-18.515,1,0.0363,181.740,0.143
1024,indie,Vampire Weekend,Big Blue,62,0.8180,0.691,108573,0.3060,0.007980,0.0992,-9.650,1,0.0631,149.965,0.413


In [ ]:
print(df.columns)

Index(['type', 'artist', 'song_title', 'popularity', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'valence'],
      dtype='object')


In [ ]:
df.drop(['artist', 'song_title', 'popularity', 'acousticness','duration_ms','instrumentalness', 'liveness','loudness', 'mode', 'speechiness','valence'], axis=1, inplace=True)

In [ ]:
# split into input and output columns
X, y = df.values[:, 1:], df.values[:, 0]

In [ ]:
print(df.columns)

Index(['type', 'danceability', 'energy', 'tempo'], dtype='object')


In [ ]:
# ensure all data are floating point values
X = X.astype('float32')

In [ ]:
X

array([[  0.889,   0.596,  97.994],
       [  0.793,   0.455, 125.921],
       [  0.669,   0.576,  79.042],
       ...,
       [  0.657,   0.376,  84.905],
       [  0.79 ,   0.49 , 140.202],
       [  0.653,   0.468,  96.931]], dtype=float32)

In [ ]:
y

array(['hiphop', 'hiphop', 'hiphop', ..., 'indie', 'indie', 'indie'],
      dtype=object)

In [ ]:
# encode strings to integer
y = LabelEncoder().fit_transform(y)

In [ ]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



(939, 3) (463, 3) (939,) (463,)



# Model

In [ ]:
# determine the number of input features
n_features = X_train.shape[1]
n_features


3

In [ ]:
# define model
model = Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(2, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)


# Tunning

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=3, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

(939, 3) (463, 3) (939,) (463,)
Best: 0.619808 using {'batch_size': 10, 'epochs': 50}
0.529286 (0.047412) with: {'batch_size': 10, 'epochs': 10}
0.619808 (0.050381) with: {'batch_size': 10, 'epochs': 50}
0.538871 (0.035989) with: {'batch_size': 10, 'epochs': 100}
0.534611 (0.013130) with: {'batch_size': 20, 'epochs': 10}
0.548456 (0.024238) with: {'batch_size': 20, 'epochs': 50}
0.522897 (0.031340) with: {'batch_size': 20, 'epochs': 100}
0.514377 (0.019695) with: {'batch_size': 40, 'epochs': 10}
0.534611 (0.023526) with: {'batch_size': 40, 'epochs': 50}
0.589989 (0.017756) with: {'batch_size': 40, 'epochs': 100}
0.544196 (0.010543) with: {'batch_size': 60, 'epochs': 10}
0.501597 (0.021352) with: {'batch_size': 60, 'epochs': 50}
0.596379 (0.033945) with: {'batch_size': 60, 'epochs': 100}
0.485623 (0.019695) with: {'batch_size': 80, 'epochs': 10}
0.495208 (0.054469) with: {'batch_size': 80, 'epochs': 50}
0.565495 (0.074563) with: {'batch_size': 80, 'epochs': 100}
0.497338 (0.024238) with

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=3, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.520965 using {'optimizer': 'Adamax'}
0.499527 (0.021715) with: {'optimizer': 'SGD'}
0.489730 (0.014917) with: {'optimizer': 'RMSprop'}
0.489730 (0.014917) with: {'optimizer': 'Adagrad'}
0.495078 (0.017429) with: {'optimizer': 'Adadelta'}
0.473671 (0.012602) with: {'optimizer': 'Adam'}
0.520965 (0.003519) with: {'optimizer': 'Adamax'}
0.504922 (0.017429) with: {'optimizer': 'Nadam'}


In [ ]:
# Use scikit-learn to grid search the learning rate and momentum p.s. copy and pasted my edit cause programmer lazy stuff.
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=3, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.502208 using {'learn_rate': 0.01, 'momentum': 0.9}
0.482600 (0.005024) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.482600 (0.005024) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.482600 (0.005024) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.482600 (0.005024) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.489730 (0.014917) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.489730 (0.014917) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.482600 (0.005024) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.482600 (0.005024) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.482600 (0.005024) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.482600 (0.005024) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.482600 (0.005024) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.502208 (0.017976) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.482600 (0.005024) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.482600 (0.005024) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.482600 (0.005024) with: {'learn_rate': 

In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=3, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.674413 using {'init_mode': 'he_uniform'}
0.612859 (0.017502) with: {'init_mode': 'uniform'}
0.571774 (0.029136) with: {'init_mode': 'lecun_uniform'}
0.642268 (0.014605) with: {'init_mode': 'normal'}
0.552121 (0.051814) with: {'init_mode': 'zero'}
0.649458 (0.032609) with: {'init_mode': 'glorot_normal'}
0.648509 (0.020623) with: {'init_mode': 'glorot_uniform'}
0.623542 (0.011061) with: {'init_mode': 'he_normal'}
0.674413 (0.025017) with: {'init_mode': 'he_uniform'}


In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=3, activation=activation))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.643212 using {'activation': 'sigmoid'}
0.576331 (0.069083) with: {'activation': 'softmax'}
0.642261 (0.047277) with: {'activation': 'softplus'}
0.641410 (0.016093) with: {'activation': 'softsign'}
0.640540 (0.059745) with: {'activation': 'relu'}
0.578857 (0.072279) with: {'activation': 'tanh'}
0.643212 (0.032419) with: {'activation': 'sigmoid'}
0.542475 (0.080932) with: {'activation': 'hard_sigmoid'}
0.529945 (0.048329) with: {'activation': 'linear'}


In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.constraints import MaxNorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=3, kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.685128 using {'dropout_rate': 0.0, 'weight_constraint': 1}
0.685128 (0.047945) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.631546 (0.031089) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.674394 (0.032879) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.659252 (0.018227) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.661885 (0.027950) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.680664 (0.037024) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.583411 (0.029590) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.665493 (0.034159) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.647644 (0.028053) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.636053 (0.013970) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.626211 (0.022356) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.624438 (0.008460) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.665505 (0.023955) with: {'dropout_rate': 0.2, 'weight_constraint': 

In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.constraints import MaxNorm
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=3, kernel_initializer='lecun_uniform', activation='linear', kernel_constraint=MaxNorm(5)))
	model.add(Dropout(0.0))
	model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.664602 using {'neurons': 20}
0.595929 (0.043548) with: {'neurons': 1}
0.621810 (0.035089) with: {'neurons': 5}
0.630722 (0.032388) with: {'neurons': 10}
0.595929 (0.036914) with: {'neurons': 15}
0.664602 (0.017335) with: {'neurons': 20}
0.625355 (0.033813) with: {'neurons': 25}
0.619057 (0.042898) with: {'neurons': 30}


In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.constraints import MaxNorm
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(15, input_dim=3, kernel_initializer='lecun_uniform', activation='linear', kernel_constraint=MaxNorm(5)))
	model.add(Dropout(0.0))
	model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
	# Compile model
	model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.3),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/spotify_classifier.csv')

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

history = model.fit(
    X_train_scaled,
    y_train,
    batch_size=10,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.LearningRateScheduler(
            lambda epoch: 1e-3 * 10 ** (epoch / 30)
        )
    ]
)

# Evaluate

In [ ]:
# determine the number of input features
n_features = X_train.shape[1]
n_features

# define model
model = Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(2, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)


In [ ]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)
# make a prediction
row = [0.772,	0.295,	89.009]
yhat = model.predict([row])
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

Test Accuracy: 0.570
1/1 [==============================] - 0s 59ms/step
Predicted: [[0.8200614  0.17993857]] (class=0)
